In [1]:
import os, sys
# os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [11]:
dfDrivers = prodFetch("""
                      select id, status, zoneId, penaltyWallet/100 as "cummulativePenalty"
                      from drivers
                      where deletedAt is null
                      and id like 'D%'
                      """)

dfDrivers

,id,status,zoneId,cummulativePenalty
0,D0003,active,NWD03,5040.000000
1,D0005,inactive,NWD03,0.000000
2,D0009,active,NWD03,0.000000
3,D0010,active,NWD03,5100.000000
4,D0011,active,NWD03,5040.000000
...,...,...,...,...
87606,D99994,active,PSK02,0.000000
87607,D99995,active,PSK02,0.000000
87608,D99996,active,PSK02,0.000000
87609,D99997,active,PSK02,0.000000


In [12]:
dfTickets = adbFetch("""
                     select issue_id, 
    complainant_id, 
    date, 
    complainant_zone, 
    issue_name, 
    category_id, 
    category_name
FROM ticketModelNew
WHERE 
    month in ('2025-01')
    AND complainant_type = 'driver'
ORDER BY date ASC;
                     """)

dfTickets

,issue_id,complainant_id,date,complainant_zone,issue_name,category_id,category_name
0,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History
1,1881,D78257,2025-01-01,LKR01,Penalty / Leave Information,64,Swap History
2,1881,D88828,2025-01-01,LKR01,Penalty / Leave Information,64,Swap History
3,1968,D105672,2025-01-01,DNG02,Khata Balance - Information,50,Payment Related
4,2057,D106886,2025-01-01,LKR01,Less Range: Complaint within 1 hour of Swap,68,Battery
...,...,...,...,...,...,...,...
111797,2439,D6915,2025-01-31,LKR01,Penalty / Leave Information,64,Swap History
111798,2523,D87460,2025-01-31,LKR03,Less Range: Complaint within 2 hour of Swap,68,Battery
111799,2449,D69886,2025-01-31,DNG02,Station Closed: Outside Operational Hours,73,Swap Issue
111800,2440,D106162,2025-01-31,LKR03,Swap Information,64,Swap History


In [13]:
dfTxns = prodFetch("""
                   SELECT distinct driverId, date as TxnDate
                   FROM transactions
                   WHERE driverId LIKE 'D%'
                   AND deletedAt IS NULL
                   and date >= '2025-01-01'
                   and date <= '2025-01-31'
                   """)

dfTxns

,driverId,TxnDate
0,D54301,2025-01-01
1,D80106,2025-01-01
2,D72125,2025-01-01
3,D32518,2025-01-01
4,D96423,2025-01-01
...,...,...
1169521,D78880,2025-01-31
1169522,D91910,2025-01-31
1169523,D89823,2025-01-31
1169524,D59406,2025-01-31


In [14]:
dfLeaves = prodFetch("""
                     SELECT driverId, date as LeaveDate
                     FROM driverLeaves
                     WHERE driverId LIKE 'D%'
                     and deletedAt is NULL
                     and date >= '2025-01-01'
                     and date <= '2025-01-31'
                     """)

dfLeaves

,driverId,LeaveDate
0,D0003,2025-01-01
1,D0010,2025-01-01
2,D0011,2025-01-01
3,D0012,2025-01-01
4,D0015,2025-01-01
...,...,...
815700,D69090,2025-01-31
815701,D69091,2025-01-31
815702,D69092,2025-01-31
815703,D110559,2025-01-31


In [15]:
dfMerged = dfTickets.merge(dfTxns, how="left", left_on="complainant_id", right_on="driverId").drop("driverId", axis=1)
dfMerged

,issue_id,complainant_id,date,complainant_zone,issue_name,category_id,category_name,TxnDate
0,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01
1,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-02
2,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-03
3,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-04
4,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-05
...,...,...,...,...,...,...,...,...
2127109,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-27
2127110,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-28
2127111,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-29
2127112,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-30


In [16]:
dfMerged = dfMerged.merge(dfLeaves, how="left", left_on="complainant_id", right_on="driverId").drop("driverId", axis=1)
dfMerged = dfMerged.merge(dfDrivers, how = "left", left_on="complainant_id", right_on="id").drop("id", axis=1)

dfMerged

,issue_id,complainant_id,date,complainant_zone,issue_name,category_id,category_name,TxnDate,LeaveDate,status,zoneId,cummulativePenalty
0,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-07,inactive,SHG03,0.000000
1,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-09,inactive,SHG03,0.000000
2,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-14,inactive,SHG03,0.000000
3,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-15,inactive,SHG03,0.000000
4,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-16,inactive,SHG03,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
11570067,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-10,active,SHG01,0.000000
11570068,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-11,active,SHG01,0.000000
11570069,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-12,active,SHG01,0.000000
11570070,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-15,active,SHG01,0.000000


In [17]:
# Create TxnFlag and LeaveFlag
dfMerged['TxnFlag'] = np.where(dfMerged['date'] == dfMerged['TxnDate'], 1, 0)
dfMerged['LeaveFlag'] = np.where(dfMerged['date'] == dfMerged['LeaveDate'], 1, 0)

# Create InactiveFlag (if neither transaction nor leave exists on the ticket date)
dfMerged['InactiveFlag'] = np.where((dfMerged['TxnFlag'] == 0) & (dfMerged['LeaveFlag'] == 0), 1, 0)

# Categorize as Ops, NonOps, and Active
dfMerged['Category'] = np.where(dfMerged['TxnFlag'] == 1, 'Ops', 
                                np.where(dfMerged['LeaveFlag'] == 1, 'NonOps', 'Inactive'))

# Combine Ops and NonOps into Active
dfMerged['Active'] = dfMerged['Category'].isin(['Ops', 'NonOps']).astype(int)

In [18]:
zonewise_count = dfMerged.groupby(['complainant_zone', 'date', 'Category'])['complainant_id'].nunique().unstack(fill_value=0)

# Add the Active column as the sum of Ops and NonOps
zonewise_count['Active'] = zonewise_count['Ops'] + zonewise_count['NonOps']

In [19]:
dfMerged

,issue_id,complainant_id,date,complainant_zone,issue_name,category_id,category_name,TxnDate,LeaveDate,status,zoneId,cummulativePenalty,TxnFlag,LeaveFlag,InactiveFlag,Category,Active
0,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-07,inactive,SHG03,0.000000,1,0,0,Ops,1
1,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-09,inactive,SHG03,0.000000,1,0,0,Ops,1
2,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-14,inactive,SHG03,0.000000,1,0,0,Ops,1
3,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-15,inactive,SHG03,0.000000,1,0,0,Ops,1
4,1880,D34533,2025-01-01,SHG03,Swap Information,64,Swap History,2025-01-01,2025-01-16,inactive,SHG03,0.000000,1,0,0,Ops,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11570067,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-10,active,SHG01,0.000000,1,0,0,Ops,1
11570068,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-11,active,SHG01,0.000000,1,0,0,Ops,1
11570069,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-12,active,SHG01,0.000000,1,0,0,Ops,1
11570070,2439,D44873,2025-01-31,SHG01,Penalty / Leave Information,64,Swap History,2025-01-31,2025-01-15,active,SHG01,0.000000,1,0,0,Ops,1


In [24]:
zonewise_count = zonewise_count.reset_index()

zonewise_count = zonewise_count.rename(columns={'complainant_zone': 'Zone', 'date': 'Date'})

zonewise_count

WorksheetNotFound()


Category,Zone,Date,Inactive,NonOps,Ops,Active
0,BHR01,2025-01-01,89,23,58,81
1,BHR01,2025-01-02,82,10,60,70
2,BHR01,2025-01-03,76,19,45,64
3,BHR01,2025-01-04,69,12,38,50
4,BHR01,2025-01-05,53,14,29,43
...,...,...,...,...,...,...
1480,SHG03,2025-01-27,172,21,131,152
1481,SHG03,2025-01-28,155,24,119,143
1482,SHG03,2025-01-29,279,96,124,220
1483,SHG03,2025-01-30,150,37,105,142


In [25]:
write("https://docs.google.com/spreadsheets/d/1pnRYP1kIaEfSa7VEVuk69VcZS4Z_ajnhLcgOeqy2IjM/edit?gid=487687816#gid=487687816", "tickets", zonewise_count)

Written successfully.
